<a href="https://colab.research.google.com/github/Ihsan1331/LLMs-Open-Course-by-Kalbe-Ahmad-Ihsan-Farhani/blob/main/1%20Hacking%20the%20HuggingFace%20Transformers%20Library/Text_Classification_ChatGPT_Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Install Packages
---
course link: https://huggingface.co/docs/transformers/tasks/sequence_classification#evaluate

ChatGPT link: https://chat.openai.com/share/aa4d86c2-b8eb-4a62-8817-f2605e69aceb

In [ ]:
!pip install datasets -q
!pip install transformers[torch] -q
!pip install accelerate -U -q
!pip install evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00


#1. Load the Dataset

In [ ]:
from datasets import load_dataset

# Load the IMDB dataset
imdb_dataset = load_dataset('imdb')

# Access the train and test splits
train_dataset = imdb_dataset['train']
test_dataset = imdb_dataset['test']

# Display the first few examples
print("Training Samples:")
print(train_dataset[0])
print(train_dataset[1])
print(train_dataset[2])

print("\nTesting Samples:")
print(test_dataset[0])
print(test_dataset[1])
print(test_dataset[2])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Training Samples:
{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes 

#2. Preprocessing

In [ ]:
from transformers import DistilBertTokenizerFast
from datasets import load_dataset

# Load the IMDB dataset
imdb_dataset = load_dataset('imdb')

# Initialize the DistilBERT tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

# Define a function for preprocessing
def tokenize_and_process(example):
    # Tokenize the text and truncate/pad to a maximum length
    tokens = tokenizer(example['text'], max_length=512, padding="max_length", truncation=True)

    # Return the tokens along with the label
    return {'input_ids': tokens['input_ids'], 'attention_mask': tokens['attention_mask'], 'label': example['label']}

# Tokenize and process the training and testing datasets
train_dataset = train_dataset.map(tokenize_and_process, batched=True)
test_dataset = test_dataset.map(tokenize_and_process, batched=True)

# Display the first preprocessed example
print("Preprocessed Training Sample:")
print(train_dataset[0])


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Preprocessed Training Sample:
{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nu

In [ ]:
print(train_dataset[0].keys())


dict_keys(['text', 'label', 'input_ids', 'attention_mask'])


#3. Evaluation And Training

In [ ]:
from datasets import load_dataset
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast, Trainer, TrainingArguments

# Load the IMDB dataset
imdb_dataset = load_dataset('imdb')

# Initialize the DistilBERT tokenizer and model
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

# Define a function for preprocessing
def tokenize_and_process(example):
    tokens = tokenizer(example['text'], max_length=512, padding="max_length", truncation=True)
    return {'input_ids': tokens['input_ids'], 'attention_mask': tokens['attention_mask'], 'label': example['label']}

# Tokenize and process the training and testing datasets
train_dataset = imdb_dataset['train'].map(tokenize_and_process, batched=True)
test_dataset = imdb_dataset['test'].map(tokenize_and_process, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Define a function for computing accuracy
def compute_accuracy(p):
    preds = p.predictions.argmax(axis=1)
    return {"accuracy": (preds == p.label_ids).mean()}

# Set the accuracy function for evaluation
trainer.compute_metrics = compute_accuracy

# Train the model
trainer.train()


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy
1,0.225800,0.236830,0.918640
2,0.152500,0.227116,0.931720


TrainOutput(global_step=3126, training_loss=0.20231847586116192, metrics={'train_runtime': 3241.9782, 'train_samples_per_second': 15.423, 'train_steps_per_second': 0.964, 'total_flos': 6623369932800000.0, 'train_loss': 0.20231847586116192, 'epoch': 2.0})